In [1]:
import os
import shutil

main_directory = "/Users/rich/Desktop/AIMusic/Datasets/"

#move files from subfolders to main folder
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name) 
            if file_name != ".DS_Store":
                destination_path = os.path.join(main_directory, file_name)
                shutil.move(file_path, destination_path)
#del subfolders
for folder_name in os.listdir(main_directory):
    folder_path = os.path.join(main_directory, folder_name)
    if os.path.isdir(folder_path):
        shutil.rmtree(folder_path)
#rename files
count = 1
for file_name in os.listdir(main_directory):
    file_path = os.path.join(main_directory, file_name)
    if os.path.isfile(file_path):
        new_file_name = str(count) + os.path.splitext(file_name)[1]
        new_file_path = os.path.join(main_directory, new_file_name)
        os.rename(file_path, new_file_path)
        count += 1

#rename files
count = 1
for file_name in os.listdir(main_directory):
    file_path = os.path.join(main_directory, file_name)
    if os.path.isfile(file_path):
        new_file_name = str(count) + os.path.splitext(file_name)[1]
        new_file_path = os.path.join(main_directory, new_file_name)
        os.rename(file_path, new_file_path)
        count += 1

In [2]:
import pretty_midi
import numpy as np
import glob

midi_file_paths = glob.glob('/Users/rich/Desktop/AIMusic/Datasets/*.mid')

all_notes = []
all_velocities = []

for file_path in midi_file_paths:
    try:
        
        midi_data = pretty_midi.PrettyMIDI(file_path)

        notes = []
        velocities = []
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes.append(note.pitch)
                velocities.append(note.velocity)

        all_notes.extend(notes)
        all_velocities.extend(velocities)
        
    except Exception as e:
        print(f"Error processing MIDI file {file_path}: {str(e)}")

notes_array = np.array(all_notes)
velocities_array = np.array(all_velocities)



Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/49.mid: data byte must be in range 0..127


/Users/Rich/Library/Python/3.9/lib/python/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/411.mid: running status without last_status
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/375.mid: Message length 2093056 exceeds maximum length 1000000
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/406.mid: data byte must be in range 0..127
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/604.mid: data byte must be in range 0..127
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/837.mid: data byte must be in range 0..127
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/562.mid: MIDI file has a largest tick of 12884927490, it is likely corrupt
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/367.mid: data byte must be in range 0..127
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/315.mid: data byte must be in range 0..127
Error processing MIDI file /Users/rich/Desktop/AIMusic/Datasets/905.mid: data byte must be 

In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
notes_normalized = scaler.fit_transform(notes_array.reshape(-1, 1))
velocities_normalized = scaler.fit_transform(velocities_array.reshape(-1, 1))

In [4]:
# 合併音符和速度陣列
combined_data = np.concatenate((notes_normalized, velocities_normalized), axis=1)


In [5]:
sequence_length = 100  # 定義輸入序列的長度
inputs = []
targets = []

for i in range(len(combined_data) - sequence_length):
    inputs.append(combined_data[i:i+sequence_length])
    targets.append(combined_data[i+sequence_length])

inputs = np.array(inputs)
targets = np.array(targets)


In [6]:
split_index = int(0.8 * len(inputs))  # 80% 作為訓練集，20% 作為測試集

train_inputs = inputs[:split_index]
train_targets = targets[:split_index]

test_inputs = inputs[split_index:]
test_targets = targets[split_index:]


In [7]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(sequence_length, combined_data.shape[1])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(combined_data.shape[1], activation='sigmoid')
])

model.compile(optimizer='adam', loss='mse')

model.fit(train_inputs, train_targets, epochs=5, batch_size=64)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-05 19:27:02.636290: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-05 19:27:02.636747: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2023-06-05 19:27:07.719718: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-05 19:27:08.727344: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-06-05 19:27:08.919502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-06-05 19:27:09.317945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


34316/34316 [==============================] - 1263s 37ms/step - loss: 0.0076
Epoch 2/5
34316/34316 [==============================] - 1333s 39ms/step - loss: 0.0060
Epoch 3/5
34316/34316 [==============================] - 1412s 41ms/step - loss: 0.0051
Epoch 4/5
34316/34316 [==============================] - 1478s 43ms/step - loss: 0.0047
Epoch 5/5
34316/34316 [==============================] - 1433s 42ms/step - loss: 0.0044


In [ ]:
model.save('/Users/rich/Desktop/AIMusic/Models/model1')

In [9]:
# 使用訓練好的模型生成音樂序列
generated_sequence = []

# 選擇一個隨機的起始位置作為輸入序列
start_index = np.random.randint(0, len(test_inputs)-1)
input_sequence = test_inputs[start_index]

# 定義生成的音樂序列的長度
sequence_length = 100

for _ in range(sequence_length):
    # 預測下一個時間步的音符和速度
    predicted = model.predict(np.expand_dims(input_sequence, axis=0))
    generated_sequence.append(predicted[0])
    
    # 更新輸入序列，將預測的音符和速度添加到序列的末尾
    input_sequence = np.concatenate((input_sequence[1:], np.expand_dims(predicted[0], axis=0)))

# 將生成的音樂序列還原為原始範圍
generated_sequence = scaler.inverse_transform(generated_sequence)

# 進行其他後處理步驟，例如將音符和速度轉換為MIDI事件

# 將生成的音樂序列轉換為MIDI文件
# TODO: 將生成的音樂序列轉換為MIDI事件，並保存為MIDI文件


1/1 [==============================] - 0s 32ms/step


In [10]:
from music21 import stream, note, tempo

# 建立一個音樂流（stream）
music_stream = stream.Stream()

# 設定一個預設的速度
default_tempo = tempo.MetronomeMark(number=120)

# 將生成的音樂序列轉換為音符和和弦，並將它們添加到音樂流中
for sequence in generated_sequence:
    pitch = int(sequence[0])  # 假設音符在序列中的第一個位置
    duration = float(sequence[1])  # 假設持續時間在序列中的第二個位置

    n = note.Note()
    n.pitch.midi = pitch
    n.duration.quarterLength = duration

    music_stream.append(n)

# 將預設速度添加到音樂流中
music_stream.insert(0, default_tempo)

# 將音樂流保存為 MIDI 文件
midi_filename = "generated_music.mid"
music_stream.write("midi", fp=midi_filename)


'generated_music.mid'

In [11]:
import tkinter as tk
import pygame

# 初始化 pygame
pygame.init()

# 設定全局變數
is_playing = False

# 播放 MIDI 文件的函數
def play_music():
    global is_playing
    if not is_playing:
        pygame.mixer.music.play()
        is_playing = True

# 暫停 MIDI 文件的函數
def pause_music():
    global is_playing
    if is_playing:
        pygame.mixer.music.pause()
        is_playing = False

# 停止 MIDI 文件的函數
def stop_music():
    global is_playing
    if is_playing:
        pygame.mixer.music.stop()
        is_playing = False

# 創建主視窗
window = tk.Tk()

# 創建播放按鈕
play_button = tk.Button(window, text="播放", command=play_music)
play_button.pack()

# 創建暫停按鈕
pause_button = tk.Button(window, text="暫停", command=pause_music)
pause_button.pack()

# 創建停止按鈕
stop_button = tk.Button(window, text="停止", command=stop_music)
stop_button.pack()

# 載入 MIDI 文件
midi_filename = "generated_music.mid"
pygame.mixer.music.load(midi_filename)

# 開始主迴圈
window.mainloop()

# 關閉 pygame
pygame.quit()


: 

: 